In [1]:
%matplotlib inline
import numpy as np
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
from geopy.distance import vincenty

In [2]:
train_bases_fname = '../Data/Train/datamart_production_basestations_history.csv'
assert(os.path.exists(train_bases_fname))
bases_df = pd.read_csv(train_bases_fname, sep=';', low_memory=False)
bases_df.head()

,site_status,sno_name,date_valid,ai_lastedittime,ai_lastedittype,ai_type,antennagain,attenuation,attenuationdirect,attenuationindirect,...,type,type_text,version_campaignid,version_campaignstatus,version_classname,version_current,version_planned,waintingforcbit,warrantystarttime,wwanmode
0,0,SIGFOX_Spain_Cellnex,2018-01-01 00:00:00,2017-03-30 09:30:01,0,NaN,5.0,0.0,NaN,NaN,...,0,Standard,NaN,NaN,NaN,v4.3.6,NaN,f,2014-02-25 23:00:00,0
1,0,SIGFOX_Spain_Cellnex,2018-01-01 00:00:00,2017-12-07 13:36:30,0,NaN,5.0,0.0,NaN,NaN,...,0,Standard,NaN,NaN,NaN,v4.3.6,NaN,f,2015-01-11 23:00:00,1
2,0,SIGFOX_UK_Arqiva,2018-01-01 00:00:00,2017-06-19 08:45:07,1,NaN,0.0,0.0,NaN,NaN,...,0,Standard,NaN,NaN,NaN,v4.1.9,NaN,t,2014-09-08 22:00:00,1
3,0,SIGFOX_UK_Arqiva,2018-01-01 00:00:00,2017-08-11 12:57:49,0,NaN,8.0,0.0,NaN,NaN,...,0,Standard,597992443c878975c85f30f1,2.0,NaN,v4.3.6,NaN,t,2014-03-11 23:00:00,1
4,0,SIGFOX_France,2018-01-01 00:00:00,2017-08-11 09:19:38,0,NaN,5.0,0.0,NaN,NaN,...,0,Standard,NaN,NaN,NaN,v4.3.6,NaN,f,2014-10-22 22:00:00,1


In [3]:
drop_columns = ['ai_type','power','preamp2','version_classname','attenuationdirect',
               'attenuationindirect', 'date_valid', 'ai_lastedittime', 'ai_lastedittime',
               'commtime', 'decommtime', 'creationtime', 'firstcommtime', 'restarttime', 
               'warrantystarttime', 'version_campaignid', 'version_campaignstatus',
               'version_current', 'version_planned', 'cavityfilterversion',
               'cavityfilterversion_text', 'cavityfilterversiondescription',
               'lifecyclestatus', 'lifecyclestatus_text', 'keepalive', 'exploitationinfo',
               'exported', 'installer', 'ip', 'last', 'lastping', 'name', 'simplesite',
               'siteid', 'objid', 'description', 'diagnosticmode', 'lnabypass', 'downtime',
               'preamp1', 'ramlog', 'bitrate', 'servicecoverage']
"""
some_columns = ['site_status', 'sno_name', 'ai_lastedittype', 'bitrate',
       'capabilities', 'conn_text', 'conns', 'dead',
       'downlinkauthorized',
       'downlinkenabled',
       'gainflag', 'globalcoverageenable', 'groupid',
       'hwversion', 'lna_version',
       'macrochannel',
       'makercode', 'mastequipment', 'mastequipmentdescription', 'model',
       'muted', 'noncompliantactivity', 'preamp1', 'protocol', 'ramlog',
       'servicecoverage_text', 'state',
       'state_text', 'type', 'type_text', 'waintingforcbit',
       'wwanmode']
"""
bases_df = bases_df.drop(columns = drop_columns)
bases_df.groupby(['bsid_hex']).nunique()
bases_df.drop_duplicates(['bsid_hex'], inplace=True)
bases_df.fillna(value = bases_df.median(), inplace = True)

In [4]:
#bases_df_normalized = bases_df_processed
#tmp = bases_df_processed[bases_df_processed.columns.difference(['bsid_hex'])]
#bases_df_normalized[bases_df_processed.columns.difference(['bsid_hex'])] = (tmp - tmp.mean())/tmp.std()


In [5]:
bases_df.columns

Index(['site_status', 'sno_name', 'ai_lastedittype', 'antennagain',
       'attenuation', 'azimuth', 'basefreq', 'bsid_hex', 'cableloss',
       'capabilities', 'conn', 'conn_text', 'conns', 'dead', 'dlkcf',
       'dlkgain', 'downlinkauthorized', 'downlinkenabled', 'elevation',
       'envloss', 'fields', 'gainflag', 'globalcoverageenable', 'groupid',
       'gsmoptions_delay', 'gsmoptions_phonenumber', 'hwversion', 'lat',
       'lna_version', 'lnagain', 'lnanoisefigure', 'lng', 'macrochannel',
       'makercode', 'mastequipment', 'mastequipmentdescription', 'model',
       'muted', 'noncompliantactivity', 'protocol', 'servicecoverage_text',
       'splatradius', 'state', 'state_text', 'tilt', 'type', 'type_text',
       'waintingforcbit', 'wwanmode'],
      dtype='object')

In [6]:
bases_df.dtypes

site_status                   int64
sno_name                     object
ai_lastedittype               int64
antennagain                 float64
attenuation                 float64
azimuth                     float64
basefreq                      int64
bsid_hex                     object
cableloss                   float64
capabilities                 object
conn                        float64
conn_text                    object
conns                        object
dead                         object
dlkcf                         int64
dlkgain                       int64
downlinkauthorized           object
downlinkenabled              object
elevation                     int64
envloss                     float64
fields                        int64
gainflag                     object
globalcoverageenable         object
groupid                      object
gsmoptions_delay            float64
gsmoptions_phonenumber      float64
hwversion                    object
lat                         

In [7]:
bases_df[['dlkcf', 'dlkgain', 'downlinkauthorized', 'downlinkenabled',
       'elevation', 'envloss', 'fields', 'gainflag', 'globalcoverageenable',
       'groupid', 'gsmoptions_delay', 'gsmoptions_phonenumber', 'hwversion',
       'lat', 'lna_version', 'lnagain', 'lnanoisefigure', ]].describe()

,dlkcf,dlkgain,elevation,envloss,fields,gsmoptions_delay,gsmoptions_phonenumber,lat,lnagain,lnanoisefigure
count,9.298000e+03,9298.000000,9298.000000,9298.000000,9.298000e+03,9298.00000,9.298000e+03,9298.000000,9298.000000,9298.000000
mean,8.682214e+08,70.532480,27.489460,3.926920,1.112837e+10,135.50000,3.725649e+10,46.161188,20.886811,4.483922
std,4.852789e+07,12.053137,17.777888,3.689458,1.571726e+11,1.53271,1.026842e+02,7.522058,1.993386,0.440362
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,31.00000,3.725648e+10,22.026200,-26.000000,0.000000
25%,8.695250e+08,64.000000,16.000000,2.000000,0.000000e+00,135.50000,3.725649e+10,42.083554,21.000000,4.500000
50%,8.695250e+08,64.000000,25.000000,3.000000,0.000000e+00,135.50000,3.725649e+10,47.863234,21.000000,4.500000
75%,8.695250e+08,64.000000,35.000000,5.000000,0.000000e+00,135.50000,3.725649e+10,50.924049,21.000000,4.500000
max,1.296100e+09,100.000000,187.000000,80.000000,2.613489e+12,240.00000,3.725650e+10,67.833628,25.000000,6.400000


In [8]:
#bases_df.to_csv('../Data/Train/basestation.csv', index=False)

In [9]:
#bases_df = pd.read_csv('../Data/Train/basestation.csv', sep=',', low_memory=True)

In [10]:
train_messages_fname = '../Data/Train/Train_dataset.csv'
assert(os.path.exists(train_messages_fname))
messages_df = pd.read_csv(train_messages_fname, sep=',', low_memory=False)
messages_df.head()

,Unnamed: 0,messageid,latitude,longitude,did,did_hex,time_msg,time_ux,time_ux_client,motion,...,radius,datepart,seqnumber,dtid,nseq,rssi,bsid_hex,bsid,snr,freq
0,15234854,5a74b2a8e541cd137cef7232,48.969345,2.761700,5138603,4E68AB,2018-02-02 18:49:12,1517597352000,1.517597e+12,t,...,NaN,2018-02-02,3197,5491a93b9336908c3b1b4c70,0,-135,2119,8473,16.100000,868049261
1,12406538,5b216b5ae541cd3845b8c2c5,48.984978,2.361960,5132655,4E516F,2018-06-13 19:07:06,1528916826000,1.528917e+12,t,...,NaN,2018-06-13,835,5491a93b9336908c3b1b4c70,0,-134,2035,8245,10.890000,868192913
2,9069667,5addfec9e541cd123f0bd3f5,48.837357,2.483158,5140851,4E7173,2018-04-23 15:42:01,1524498121000,1.524498e+12,f,...,NaN,2018-04-23,103,5491a93b9336908c3b1b4c70,0,-104,3ED2,16082,47.549999,868134621
3,7404394,5a74bdcf12f14352d21e92a9,48.862305,2.366602,5136312,4E5FB8,2018-02-02 19:36:47,1517600207000,1.517600e+12,f,...,NaN,2018-02-02,570,5491a93b9336908c3b1b4c70,0,-106,DEA,3562,45.070000,868139828
4,11716433,5b00b43acf554f398fe235dd,48.752403,2.495280,5138934,4E69F6,2018-05-19 23:33:14,1526772794000,1.526773e+12,f,...,NaN,2018-05-19,3079,5491a93b9336908c3b1b4c70,0,-128,DD9,3545,14.440000,868066447


In [11]:
import datetime

def datesplit(originalDate):
    originalDate = str(originalDate)
    year = int(originalDate[0:4])
    month = int(originalDate[5:7])
    day = int(originalDate[8:10])
    hour = int(originalDate[11:13])
    minute = int(originalDate[14:16])
    second = int(originalDate[17:19])
    return datetime.datetime(year,month,day,hour,minute,second)

In [12]:
messages_df["year"]=messages_df["time_msg"].apply(lambda x:datesplit(x).year)
messages_df["month"]=messages_df["time_msg"].apply(lambda x:datesplit(x).month)
messages_df["weekday"]=messages_df["time_msg"].apply(lambda x:datesplit(x).weekday())
messages_df["day"]=messages_df["time_msg"].apply(lambda x:datesplit(x).day)
messages_df["hour"]=messages_df["time_msg"].apply(lambda x:datesplit(x).hour)
messages_df["minute"]=messages_df["time_msg"].apply(lambda x:datesplit(x).minute)
messages_df["second"]=messages_df["time_msg"].apply(lambda x:datesplit(x).second)

In [13]:
drop_columns = ['Unnamed: 0', 'time_msg','altitude', 'nb_satellites', 'year',
          'messageid', 'datepart', 'bsid',
           'data_type', 'did_hex']
messages_df.drop(columns = drop_columns, inplace = True)
messages_df.fillna(value = messages_df.median(), inplace = True)

In [14]:
messages_df = messages_df.join(bases_df.set_index('bsid_hex'), on='bsid_hex')

In [15]:
messages_df.dropna( subset=['time_ux'] ,inplace=True)
messages_df = messages_df.sample(frac=1).reset_index(drop=True)
#messages_df = pd.get_dummies(messages_df, columns=['bsid_hex'], dummy_na=True)

In [16]:
messages_df.to_csv('../Data/Train/Train_dataset_2.csv', index=False)

In [2]:
messages_df = pd.read_csv('../Data/Train/Train_dataset_2.csv', sep=',', low_memory=False)

In [3]:
some_columns=['site_status', 'sno_name', 'ai_lastedittype',
        'capabilities', 'conn_text', 'conns', 'dead',
        'downlinkauthorized',
        'downlinkenabled',
        'gainflag', 'globalcoverageenable', 'groupid',
        'hwversion', 'lna_version',
        'macrochannel',
        'makercode', 'mastequipment', 'mastequipmentdescription', 'model',
        'muted', 'noncompliantactivity', 'protocol',
        'servicecoverage_text', 'state',
        'state_text', 'type', 'type_text', 'waintingforcbit',
        'wwanmode',
        'motion', 'dtid', 'month', 'weekday', 'day', 'hour', 'minute', 
        'second', 'speed', 'rssi', 'nseq', 'bsid_hex']

messages_df = pd.get_dummies(messages_df, columns=some_columns, dummy_na=True)

MemoryError: 

In [ ]:
#messages_df_normalized = messages_df
#tmp = messages_df[messages_df.columns.difference(['bsid_hex'])]
#messages_df_normalized[messages_df.columns.difference(['bsid_hex'])] = (tmp - tmp.mean())/tmp.std()

In [ ]:
#messages_df = messages_df.join(bases_df.set_index('bsid_hex'), on='bsid_hex')

In [ ]:
#messages_df.nunique()

In [ ]:
#messages_df.to_csv('../Data/Train/Train_dataset_3.csv', index=False)

In [ ]:
print(messages_df.dtypes)
print(messages_df.shape)

In [ ]:
#messages_df_processed.describe()

In [ ]:
col = ['latitude',
'longitude',
'did',
'time_ux',
'time_ux_client',            
'radius',                    
'seqnumber',               
'snr',                      
'freq',                       
'antennagain',               
'attenuation',             
'azimuth',                   
'basefreq',                   
'cableloss',                 
'conn',                      
'dlkcf',                      
'dlkgain',                    
'elevation',                  
'envloss',                  
'fields',                     
'gsmoptions_delay',          
'gsmoptions_phonenumber',   
'lat',                       
'lnagain',                  
'lnanoisefigure',            
'lng',                      
'servicecoverage',           
'splatradius',
'tilt']

mean = messages_df[col].mean()
std = messages_df[col].std()

name_mean = ('../Data/Train/mean.csv')
mean.to_csv(name_mean, index=False)

name_std = ('../Data/Train/std.csv')
std.to_csv(name_std, index=False)


In [ ]:
messages_df[col].describe()

In [ ]:
messages_df[col] = (messages_df[col]-mean)/std

In [ ]:
messages_df.head()

In [ ]:
N = 100
batch = math.floor(1.333141e+07/N)
for k in range(N):
    name = ('../Data/Train/Tensorflow/Train_dataset_%d.csv' % k)
    messages_df[k*batch:(k+1)*batch].to_csv(name, index=False)